# RED SOCIAL (CONSULTAS) #


**Utilizando la BD 'Social' **

In [1]:
USE SOCIAL

Commands completed successfully.

Total execution time: 00:00:00.385

# Basic SELECT, Table Variables, Set Operators

**1. Friends of Gabes: : Find the names of all students who are friends with someone named Gabriel.**

In [2]:
SELECT h1.name,h2.name FROM Friend 
 INNER JOIN  Highschooler h1 on Friend.ID1 = h1.ID
 INNER JOIN Highschooler h2 on Friend.ID2 = h2.ID
 WHERE H1.name='Gabriel'

 select distinct H2.name from Highschooler H1, Friend F1, Highschooler H2 
    where H1.ID = F1.ID1 and H1.name = 'Gabriel' 
    and H2.ID = F1.ID2;


(5 rows affected)

Total execution time: 00:00:00.911

name,name
Gabriel,Jordan
Gabriel,Cassandra
Gabriel,Andrew
Gabriel,Alexis
Gabriel,Jessica


**2.  Like, scandalous!: For every student who likes someone 2 or more grades below their own grade, return that student's name and grade, and the name and grade of the student they like.**

In [16]:
select h1.name as [Student 1 Name], h1.grade as [Student 1 Grade],h2.name as [Student 2 Name], h2.grade as [Student 2 Grade] 
    from Highschooler h1, Likes L1, Highschooler h2 
        where h1.ID = L1.ID1 and h2.ID = L1.ID2 and (h1.grade-h2.grade)>=2;



(1 row affected)

(1 row affected)

Total execution time: 00:00:00.112

Student 1 Name,Student 1 Grade,Student 2 Name,Student 2 Grade
John,12,Haley,10


Student 1 Name,Student 1 Grade,Student 2 Name,Student 2 Grade
John,12,Haley,10


**3.  Should totally be dating: For every pair of students who both like each other, return the name and grade of both students. Include each pair only once.**

In [18]:
select H1.name, H1.grade, H2.name, H2.grade from Highschooler H1, Highschooler H2, Likes L1, Likes L2 
where (H1.ID = L1.ID1) and (H2.ID = L1.ID2) and (L1.ID1 = L2.ID2) and (L1.ID2 = L2.ID1) and (L1.ID1 < L1.ID2)

(2 rows affected)

Total execution time: 00:00:00.051

name,grade,name,grade
Gabriel,9,Cassandra,9
Jessica,11,Kyle,12


**4.  Alias: For every situation where student A likes student B, but student B likes a different student C, return the names and grades of A, B, and C**

In [22]:
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade 
    from Highschooler H1, Highschooler H2, Highschooler H3, Likes L1, Likes L2 
        where (H1.ID = L1.ID1) and (H2.ID = L1.ID2) and (H3.ID = L2.ID2) and (L1.ID2 = L2.ID1) and (L1.ID1 <> L2.ID2)


(2 rows affected)

Total execution time: 00:00:00.469

name,grade,name,grade,name,grade
Andrew,10,Cassandra,9,Gabriel,9
Gabriel,11,Alexis,11,Kris,10


**5.  Need more intel: Find all students who do not appear in the Likes table (as a student who likes or is liked) and return their names and grades. Sort by grade, then by name within each grade. (Hint: Without subqueries, you will need to use the EXCEPT operator.)**

In [24]:
select name, grade from Highschooler H1
     where H1.ID not in (select ID1 from Likes union select ID2 from Likes) order by grade, name;


(3 rows affected)

Total execution time: 00:00:00.122

name,grade
Jordan,9
Tiffany,9
Logan,12


# Subqueries in WHERE, FROM, SELECT#


**6.  Have a chance: For every situation where student A likes student B, but we have no information about whom B likes (that is, B does not appear as an ID1 in the Likes table), return A and B's names and grades.**

In [25]:
select H1.name, H1.grade, H2.name, H2.grade from Highschooler H1, Highschooler H2, Likes L1 where H1.ID = L1.ID1 and H2.ID = L1.ID2 and L1.ID2 not in (select ID1 from Likes);


(4 rows affected)

Total execution time: 00:00:00.155

name,grade,name,grade
John,12,Haley,10
Austin,11,Jordan,12
Alexis,11,Kris,10
Brittany,10,Kris,10


**7.  Narrow-minded: Find names and grades of students who only have friends in the same grade. Return the result sorted by grade, then by name within each grade.**

In [ ]:
select distinct H1.name from Highschooler H1, Friend F1 where H1.ID = F1.ID1 and H1.grade = all(select H2.grade from Highschooler H2, Friend F2 where F2.ID1 = F1.ID1 and H2.ID = F2.ID2);

select H1.name, H1.grade from Highschooler H1 where H1.ID not in (select H2.ID from Highschooler H2, Friend F1, Highschooler H3 where H2.ID = F1.ID1 and H3.ID = F1.ID2 and H2.grade <> H3.grade) order by grade, name;


**8.  Broad-minded: Find names and grades of students who only have friends in different grades.**

In [ ]:
select distinct H1.name from Highschooler H1, Friend F1 where H1.ID = F1.ID1 and H1.grade <> all(select H2.grade from Highschooler H2, Friend F2 where F2.ID1 = F1.ID1 and H2.ID = F2.ID2);


**9.  Could you introduce me? For each student A who likes a student B where the two are not friends, find if they have a friend C in common. For all such trios, return the name and grade of A, B, and C.**

In [ ]:
select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade from Highschooler H1, Highschooler H2, Highschooler H3, Likes L1, Friend F1, Friend F2 where H1.ID = L1.ID1 and H2.ID = L1.ID2 and (H1.ID, H2.ID) not in (select ID1, ID2 from Friend) and (F1.ID1 = H3.ID and F1.ID2 = H1.ID and F2.ID1 = H3.ID and F2.ID2 = H2.ID);

select H1.name, H1.grade, H2.name, H2.grade, H3.name, H3.grade from Highschooler H1, Highschooler H2, Highschooler H3, Likes L1, Friend F1, Friend F2 where H1.ID = L1.ID1 and H2.ID = L1.ID2 and F1.ID1 = H3.ID and F1.ID2 = H1.ID and F2.ID1 = H3.ID and F2.ID2 = H2.ID and H1.ID not in (select ID1 from Friend where ID2 = H2.ID);


# Aggregation #

**10.  Duplication: Find the difference between the number of students in the school and the number of different first names.**

In [ ]:
select count(ID) - count(distinct name) from Highschooler;

**11.  Average affability: What is the average number of friends per student?**

In [ ]:
select avg(friends) from (select ID1,count(ID2) as friends from Friend group by ID1) G;


**12.  Social circle: Find the number of students who are either friends with Cassandra or are friends of friends of Cassandra. Do not count Cassandra, even though technically she is a friend of a friend.**

In [ ]:
create table t1 as (select ID2 from Friend where ID1 = (select ID from Highschooler where name = 'Cassandra'));

create table t2 as select distinct ID2 from Friend where ID1 in (select * from t1)and ID2 <> (select ID from Highschooler where name = 'Cassandra');

select count(ID2) from ((select * from t1) union (select * from t2)) G;

**13.  It must be their brains, not their looks: Find the name and grade of all students who are liked by more than one other student.**

In [ ]:
select name, grade from Highschooler where ID in (select ID2 from (select count(ID1) as liked,ID2 from Likes group by ID2) G where G.liked >= 2);


**14.  Most gregarious: Find the name and grade of the student(s) with the greatest number of friends.**

In [ ]:
create table t1 as select ID1, count(ID2) as nof from Friend group by ID1;

select name, grade from Highschooler where ID in (select ID1 from t1 where t1.nof = (select max(nof) from t1));


# Data Modification Statements#

Each of the questions in this section (except the first) assumes the database is in the state after the modifications from the previous question. If you find you've made a mistake or want to restart, just reload the schema and data from scratch.

**15.  End of the school year: Increment the year of all students by 1, such that all 9th graders become 10th graders, all 10th graders become 11th graders, and so on. To check your modification, find the names of all students who are in a grade higher than 12 (we'll deal with them later).**

In [ ]:
update Highschooler set grade = grade + 1;

**16.  Graduating seniors: After the previous update, some students ended up in 13th grade. They're actually graduating, so remove all tuples in Highschooler with a grade higher than 12. To check your modification, find the total number of students remaining in the Highschooler table.**

In [ ]:
delete from Highschooler where grade > 12;


**17.  Now clean up this mess: While friendships may continue after graduation, we can't model them for students no longer in our database. Remove all tuples in Friend and all tuples in Likes that refer to nonexistent students. (This will require two separate commands.) To check your modification, find the names of all students whom Austin likes or who are friends with Austin.**

In [ ]:
delete from Friend where ID1 not in (select ID from Highschooler) and ID2 not in (select ID from Highschooler);
delete from Likes where ID1 not in (select ID from Highschooler) and ID2 not in (select ID from Highschooler);
select H2.name from Highschooler H1, Friend, Highschooler H2 where H1.name = 'Austin' and H1.ID = Friend.ID1 and H2.ID = Friend.ID2;


**18.  Summertime: Now that they're on vacation, our students have more time to spend with one another. As a result, their social circles have expanded: For all cases where A is friends with B, and B is friends with C, add a new friendship for the pair A and C. (Do not add duplicate friendships, friendships that already exist, or friendships with oneself.) To check your modification, for the now 10th-grader Jordan, return the names of all her friends.**

In [ ]:
insert into Friend select F1.ID1, F2.ID2 from Friend F1, Friend F2 where F1.ID2 = F2.ID1 and F1.ID1 <> F2.ID2 and (F1.ID1, F2.ID2) not in (select * from Friend);
select H2.name from Highschooler H1, Friend, Highschooler H2 where H1.name = 'Jordan' and H1.ID = Friend.ID1 and H2.ID = Friend.ID2;+-----------+
